# Redundant Internal Coordinates
#### This is a tutorial for the conversion to and from internal coordinates using the B matrix.

Redundent Internal Coordiantes, as proposed by Bakken and Helgaker, are defined using stretches, bends, and dihedral angles. Including vibrational motions, there are up to 3N-6 degrees of freedom for a molecule. Defining water with internal coordinates, there are two stretches, and one bend this defining all degrees of freedom. For molecules with more than three atoms, the number of internal coordiantes quickly becomes greater than the number of degrees of freedom and thus redundant.  

In order to convert cartesian coordiantes to internal coordinates, Wilson B matrix is used. The B matrix is defined as the derivative of the internal coordinates with respect to cartesian coordinates, and has dimensions of internal coordinates by 3 * N. 

$$\textbf {B}_{ij} = \frac{\delta q_i}{\delta x_j}$$

Simple rearrangment allows for the conversion between small changes in cartesian geometry to internal coordinates
$$\textbf {B}_{ij} \delta x = \delta q$$

This does not allow for the direct conversion between the geometry in cartesians and in internals. However, this isn't particularly helpful, since our goal is usually to conver a change in internal coordinates to cartesians which is covered later in discussing the back transformation. To create the internal geometry, we need to calculate the distances and angles using simple geometry.

Lets start by defining a simple cartesian geometry for ammonia and then defining all of the internal coordinates for the molecule. Note that the number of internal coordinates is greater than the number of degrees of freedom.

In [14]:
import psi4
import numpy as np
from optking import printTools
printTools.printInit(printTools.cleanPrint)
printTools.print_opt("hi")

#a starting geometry can be taken from pubchem
mol = psi4.geometry("""
  pubchem:ammonium
""")
mol.set_multiplicity (1)
mol.set_molecular_charge(1)

# Set some options
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})
mol.update_geometry()
xyzGeom = np.array(mol.geometry())
print ("Starting Geometry for Ammonium")
print (xyzGeom)


from optking import stre, bend
r1 = stre.STRE(0,1)
r2 = stre.STRE(0,2)
r3 = stre.STRE(0,3)
r4 = stre.STRE(0,4)
b1 = bend.BEND(1,0,2)
b2 = bend.BEND(2,0,3)
b3 = bend.BEND(3,0,4)
b4 = bend.BEND(4,0,1)
b5 = bend.BEND(4,0,2)
b6 = bend.BEND(3,0,1)
intcos = (r1, r2, r3, r4, b1, b2, b3, b4, b5, b6)

print ("\nList of internal Coordinates in Bohr and Radians")
for i in range (len(intcos)):
    #q(xyzGeom) simply calls v3d.dist() for the two atoms in a stretch, and v3d.angle() for the three atoms in a bond angle
    print (" %d %s %.16f" % (i + 1, intcos[i], intcos[i].q(xyzGeom)))

#create an array for geometry in internals
qGeom = np.zeros(len(intcos), float)
for i in range (len(intcos)):
    qGeom[i] = intcos[i].q(xyzGeom)

print("\n Geometry in internals")    
printTools.printArray(qGeom)    

hi	Searching PubChem database for ammonium
	Found 1 result
Starting Geometry for Ammonium
[[  0.00000000e+00  -8.29781462e-35   3.92826956e-06]
 [  1.58424594e+00   4.85035429e-17   1.12021175e+00]
 [  4.85021276e-17  -1.58419971e+00  -1.12023904e+00]
 [ -1.58424594e+00  -4.85035429e-17   1.12021175e+00]
 [ -4.85021276e-17   1.58419971e+00  -1.12023904e+00]]

List of internal Coordinates in Bohr and Radians
 1  R(1,2) 1.9402836773125387
 2  R(1,3) 1.9402662259545544
 3  R(1,4) 1.9402836773125387
 4  R(1,5) 1.9402662259545544
 5  B(2,1,3) 1.9106377347342236
 6  B(3,1,4) 1.9106377347342236
 7  B(4,1,5) 1.9106377347342236
 8  B(2,1,5) 1.9106377347342236
 9  B(3,1,5) 1.9105956933089669
 10  B(2,1,4) 1.9106527844269028

 Geometry in internals
   1.940284   1.940266   1.940284   1.940266   1.910638   1.910638   1.910638
   1.910638   1.910596   1.910653


Our goal is to calculate the step in internal coordintes, this requires that either the forces or gradient are converted into internals.

In order to convert between the gradient or forces we need to introduce $\delta E$ luckily this easily simplifies to the form we need

$$\frac{\textbf {B}}{\delta q} = (\delta x)^{-1} \equiv \frac{\delta q}{\delta x} \frac{\delta E}{\delta q} = \frac{\delta E}{\delta x} \equiv \textbf{B}^T g_q = g_x$$ 

First lets get the gradient from Psi4, which is in cartesian

In [17]:
psi4gradientMatrix = psi4.gradient('hf')
g = np.array( psi4gradientMatrix )
print (g)

[[ -2.95822839e-31   0.00000000e+00  -2.06186439e-06]
 [  8.05561725e-03   2.46632147e-19   5.69463262e-03]
 [  2.46334235e-19  -8.04588669e-03  -5.69360169e-03]
 [ -8.05561725e-03  -2.46632147e-19   5.69463262e-03]
 [ -2.46334235e-19   8.04588669e-03  -5.69360169e-03]]


Whereas the conversion of dispalcements into internals and gradients into cartesians was quite simple. Converting back becomes significantly easier

In order to convert gradients or dispalcements into internal cooordinates we need to implement the psuedo Inverse of the B matrix.

For gradients we start with
$$g_x = \textbf B^T f_q$$
To make B a sqaure invertible matrix we introducing the square 3N * 3N arbitrary matrix U, which is taken to be the identity matrix
$$ ug_x = u \textbf B^T f_q$$
$$ \textbf Bug_x = \textbf Bu \textbf B^T f_q$$
$$ (\textbf Bu \textbf B^T)^{-1} \textbf Bug_x = (\textbf Bu \textbf B^T)^{-1} \textbf Bu \textbf B^T f_q$$
$$ (\textbf Bu \textbf B^T)^{-1} \textbf Bug_x = f_q$$
Reducing $(\textbf Bu \textbf B^T)$ to its more common designation $\textbf G$ leaves us with
$$ \textbf G^{-1} \textbf {B}ug_x = f_q$$

The conversion of displacements into cartesians nessecitates the use of an iterative procedure called the back transformation.
As with the conversion of forces or gradients, implementation of the pseudo inverse leads to the similar looking result.
$$ \textbf G^{-1}_0 \textbf {B}^T_0 u\Delta q_0 = \Delta x_0$$

Here, the G matrix has dimensions of cartesian not internals as it is calculated with $\textbf B^Tu \textbf B$ where the u matrix has dimesnions of the number of internal coordinates

The iterative back transformation becomes nessecary because the new cartesian geometry is not perfectly eqivalent to the new geometry we stepped to in internals
$$x_0 + \Delta x = x_1$$
To converge on the correct cartesian geometry, we take the error in internals and convert again to cartesians

Call the cartesian geometry converted back to internals as previously shown, $q_x$ and the geometry we calcualted $q_1$ then
$$q_x - q_1 = q_\epsilon$$
We can then resolve our convesian equation having recalulated the nessecary matrices at the new geometry
$$ \textbf G^{-1}_n \textbf {B}_n^T u\Delta q_\epsilon = \Delta x_n$$




To demonsrtrate the need for the back transformation, we'll take a step, in internal coordinates, of 0.2 bohr for the bond lengths and then convert it into cartesians, and back again into internals

In [18]:
from optking import intcosMisc
from optking import linearAlgebra
dq0 = np.zeros(len(intcos), float)
for i in range (4):
    dq0[i] = 0.2

#This is what we should be stepping to    
qGeomWanted = np.add(qGeom, dq0)
print("New internal geometry")
printTools.printArray(qGeomWanted)

B0 = intcosMisc.Bmat(intcos, xyzGeom)

G0 = np.dot(B0.T, B0) 
#the u identity matrix is omittted above due to the dimensions
printTools.printMat(G0)    
G0inv = linearAlgebra.symmMatInv(G0, True)
u = np.identity(len(intcos))
dx0 = np.dot(G0inv, np.dot(B0.T, np.dot(u, dq0)))

print ("Displacement in cartesians")
printTools.printArray(dx0)
dx0mat = np.reshape(dx0, (5, 3))
#Compare the cartesian geometries
xyzGeom0 = np.add(dx0mat, xyzGeom)
print ("Old geometry")
print (xyzGeom)
print ("New geometry")
print(xyzGeom0)


qGeom1 = np.zeros(len(intcos), float)
for i in range (len(intcos)):
    qGeom1[i] = (intcos[i].q(xyzGeom0))
print ("New geometry with error")    
print (qGeom1)
dq1Prime = np.subtract(qGeom1, qGeom0) #This is the actual Dq we took
print ("Acutal step from cartesians")
print (dq1Prime)

Dq1 = np.subtract(dq0, dq1Prime) #This is the error in Dq

print ("This is why we need to back transformation")
print (Dq1)

New internal geometry
   2.140284   2.140266   2.140284   2.140266   1.910638   1.910638   1.910638
   1.910638   1.910596   1.910653
   2.750073   0.000000   0.000000  -0.843766  -0.000000  -0.220952  -0.531270
   0.000000   0.000000  -0.843766  -0.000000   0.220952  -0.531270   0.000000
  -0.000000
   0.000000   2.749949   0.000000  -0.000000  -0.531241  -0.000000   0.000000
  -0.843733  -0.220986  -0.000000  -0.531241   0.000000   0.000000  -0.843733
   0.220986
   0.000000   0.000000   2.750023  -0.220965  -0.000000  -0.687503   0.000000
  -0.220973  -0.687509   0.220965   0.000000  -0.687503  -0.000000   0.220973
  -0.687509
  -0.843766  -0.000000  -0.220965   0.799488   0.000000   0.283573   0.066409
  -0.022136   0.031304  -0.088539  -0.000000  -0.125216   0.066409   0.022136
   0.031304
  -0.000000  -0.531241  -0.000000   0.000000   0.398429   0.000000  -0.199220
   0.066406  -0.093909  -0.000000  -0.000000  -0.000000   0.199220   0.066406
   0.093909
  -0.220952  -0.000000  -0

NameError: name 'qGeom0' is not defined